In [ ]:
from bls_housing.pipeline.wages import build_annual_wages
from bls_housing.pipeline.duck import list_metros, get_con
import pandas as pd

years = [y for y in range(2014, 2025)] # include dates from 2014-2024
area_codes = [42660, 38900, 33460, 12420, 47900]
con = get_con()
metros = list_metros(con, area_codes)

annual_wages_df = build_annual_wages(metros, years)
print(annual_wages_df.head(10))

In [ ]:
from bls_housing.pipeline.permits import build_annual_permits

annual_permits = build_annual_permits(metros,years)
print(annual_permits.head(10))

In [ ]:
final_df = pd.merge(
    annual_wages_df,
    annual_permits,
    on=["Area", "Code", "Year"],
    suffixes=('_wage', '_permit')
)


final_df['Wage_Index'] = 1 + (final_df['Change_Real_Wage'] / 100) # adjusted for inflation
final_df['Permit_Index'] = 1 + (final_df['Change_Permit'] / 100)
final_df['Zoning_Pressure'] = final_df['Wage_Index'] / final_df['Permit_Index']

print(final_df[['Area', 'Year', 'Zoning_Pressure']].sort_values(by=['Area', 'Year']).head(10))

In [ ]:
import matplotlib.pyplot as plt
pivot_df = final_df.pivot(index='Year', columns='Area', values='Zoning_Pressure')

plt.figure(figsize=(12, 6))
for column in pivot_df.columns:
    plt.plot(pivot_df.index, pivot_df[column], marker='o', label=column)

plt.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Balanced Growth (1.0)')

plt.title('Zoning Pressure Index (2015-2024 inflation adjusted)')
plt.ylabel('Pressure (>1 means Demand outpaces Supply)')
plt.xlabel('Year')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
from pathlib import Path
# 1) merge raw totals
cumulative_df = (
    annual_wages_df[["Area", "Code", "Year", "Real_Total_Wages"]]
    .merge(
        annual_permits[["Code", "Year", "Total_Permits"]],
        on=["Code", "Year"],
        how="inner",
    )
)

# keep types sane (Year as int is cleaner than string comparisons)
cumulative_df["Year"] = cumulative_df["Year"].astype(int)

# 2) filter years
cumulative_df = cumulative_df[cumulative_df["Year"] >= 2015].copy()

# 3) attach base-year totals per Code
base_year = 2015
base = (
    cumulative_df[cumulative_df["Year"] == base_year][["Code", "Real_Total_Wages", "Total_Permits"]]
    .rename(columns={"Real_Total_Wages": "Base_Wage", "Total_Permits": "Base_Permits"})
)

dupes = base["Code"].duplicated().sum()
assert dupes == 0, f"Base year has duplicate codes: {dupes}"
cumulative_df = cumulative_df.merge(base, on="Code", how="left")

# 4) compute indices
cumulative_df["Cumul_Wage_Index"] = cumulative_df["Real_Total_Wages"] / cumulative_df["Base_Wage"]
cumulative_df["Cumul_Permit_Index"] = cumulative_df["Total_Permits"] / cumulative_df["Base_Permits"]

# 5) structural gap (guard divide-by-zero / missing base)
cumulative_df["Structural_Gap"] = cumulative_df["Cumul_Wage_Index"] / cumulative_df["Cumul_Permit_Index"]

#out_path = Path("../data/derived/cumulative_metrics.parquet")
#cumulative_df.sort_values(["Code", "Year"]).to_parquet(out_path, index=False)

In [ ]:
import matplotlib.pyplot as plt

# Pivot for plotting
pivot_gap = cumulative_df.pivot(index='Year', columns='Area', values='Structural_Gap')

plt.figure(figsize=(12, 6))

# Plot lines
for column in pivot_gap.columns:
    plt.plot(pivot_gap.index, pivot_gap[column], marker='o', linewidth=2, label=column)

# Add Reference Line (1.0 = Supply keeping up with Demand)
plt.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Balanced (1.0)')

plt.title('Cumulative Housing Deficit (Since 2015 inflation adjusted)')
plt.ylabel('Deficit Index (1.2 = Demand is 20% ahead of Supply)')
plt.xlabel('Year')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import duckdb

con = duckdb.connect(database="../data/analysis.duckdb")
con.execute("CREATE OR REPLACE TABLE annual_metrics AS SELECT * FROM final_df")
con.execute("CREATE OR REPLACE TABLE cumulative_metrics AS SELECT * FROM cumulative_df")
con.execute("CREATE OR REPLACE TABLE wages_metrics AS SELECT * FROM wages_df")
con.execute("CREATE OR REPLACE TABLE permits_metrics AS SELECT * FROM permits_df")
con.close()

In [ ]:
# Load cleaned CBSA CSV from cache instead of manual XLS parsing
from bls_housing.census_cache import load_cbsa_df
df = load_cbsa_df('2025', '1')
print(df.keys())
df.head(10)

In [ ]:
# Show cached CSV path
from bls_housing.census_cache import get_cached_csv_path
print(get_cached_csv_path('2025', '1'))


In [ ]:
df_current_area = df[df['CBSA'] == 42660]  # filter for Seattle MSA
print(df_current_area)

# get total permits for Seattle MSA
df_Seattle_total_wages = df_current_area['Total'].iloc[0]

#print(df_Seattle_total_wages.columns)
#print(df_Seattle.columns.value_counts())
print(f"[Total: {df_Seattle_total_wages}]")


In [ ]:
con.close()